In [1]:
from scipy.stats import binom, chi2
from scipy import stats
from sklearn.utils import resample

import statsmodels.api as sm
import math, statistics
import numpy as np
import pandas as pd

import matplotlib as plt
import seaborn as sns

In [2]:

def computeNoveltyRegression(variant_list):
    number_of_intervals = len(variant_list)
   
    X = getIndependentVariables(number_of_intervals)
    y = getDependentVariable(variant_list)

    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)    

    return model.rsquared


def getDependentVariable(variant_list):
    data = {"y":variant_list} 

    return pd.DataFrame(data)        


def getIndependentVariables(number_of_days):
    data = {"x1": [], "x2": []}
    for t in range(1, (number_of_days + 1)):
        data['x1'].append((1/(math.pow(t,0.35))))
        data['x2'].append((1/(math.pow(t,2)))) 

    return pd.DataFrame(data) 


def isMonotonic(variant_list):

    if len(variant_list) < 2:
        return False #no novelty effect detectable when less than 2 intervals

    if variant_list[0] < variant_list[1]:
        trend = "up"
    else:
        trend = "down"

    previous = None
    for value in variant_list:
        if previous is None:
            previous = value
        elif trend == "up" and value <= previous:
            return False
        elif trend == "down" and value >= previous:
            return False
        else:
            previous = value
    
    return True


def isMonotonic(variant_list):

    statistic, pvalue = stats.spearmanr(variant_list, list(range(len(variant_list))))
    
    print("statistic:", statistic, "p:", pvalue)
    
    return True


def noveltyTest(variant_list):

    r2 = computeNoveltyRegression(variant_list)
    print("r2:", r2)


    if r2 >= 0.8:
        if isMonotonic(variant_list):
            return True

    return False

In [3]:
single_day_lifts = [
    [1.1,1.01,1.2,1,1,1,1],
    [1,2,3,4,5,6,7,8,9,10],
    [10,9,8,7,6,5,4,3,2,1],
    [1,0,1,0,1,0,1,0,1],
    [1.3,1.2,1.25,1.1,1.2,1.1,1.05,1.05,1.05,1.05,1.05,1.05,1.05,1.05]
]

for day in single_day_lifts:
    print(day)
    is_novelty = noveltyTest(day)
    print(f"Novelty: {is_novelty}")
    print("-------")

[1.1, 1.01, 1.2, 1, 1, 1, 1]
r2: 0.9909404875502137
statistic: -0.7881104062391007 p: 0.035282699416752
Novelty: True
-------
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
r2: 0.7776200145043606
Novelty: False
-------
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
r2: 0.9367281728590601
statistic: -0.9999999999999999 p: 6.646897422032013e-64
Novelty: True
-------
[1, 0, 1, 0, 1, 0, 1, 0, 1]
r2: 0.5497579898060786
Novelty: False
-------
[1.3, 1.2, 1.25, 1.1, 1.2, 1.1, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05]
r2: 0.986250951061411
statistic: -0.8834656921986938 p: 2.8003667519988096e-05
Novelty: True
-------
